In [1]:
import pandas as pd
import os
import re
import pickle
from gensim.models import Word2Vec
from time import time

In [31]:
#loading the list of muslim related terms

In [32]:
muslim_terms = pickle.load(open("muslim_terms.pkl", "rb"))

In [39]:
#indentfying tweets about Muslims by going through every state, every day in the state, and saving the id of 
#every tweet that contains any word from our list of muslims terms

In [45]:
t = time()
state_list = os.listdir("Cleaned Data")
for state in state_list:
    t_1 = time()
    _path = os.path.join("Cleaned Data", state)
    file_list = [file for file in os.listdir(_path) if ".csv" in file]
    muslim_ids = []
    for file in file_list:
        df = pickle.load(open(os.path.join(_path, file), "rb"))
        for i in range(df.shape[0]):
            if any([term in df.at[i, "cleaned_tweet"] for term in muslim_terms]) or any([term in df.at[i, "hashtag"] for term in muslim_terms]):
                muslim_ids.append(df.at[i, "id"])
    pickle.dump(list(set(muslim_ids)), open(os.path.join(_path, "{} muslim_ids.pkl".format(state.split()[-1])), "wb"))
    print("{} completed".format(state))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2))) 

cleaned Andhra_Pradesh completed
time taken: 4.37 mins
time taken: 4.37 mins


In [48]:
#going through each state in "Cleaned Data". through each file in each state, load the file as a dataframe
#load the list of muslim ids of each state
#tag each tweet with a "1" in the newly created "muslim" column if its tweet id is in the above loaded list
#save the dataframe with the same path as the loaded file.

In [49]:
state_list = os.listdir("Cleaned Data")
t = time()
for state in state_list:
    t_1 = time()
    _path = os.path.join("Cleaned Data", state)
    muslim_ids = pickle.load(open(os.path.join(_path, "{} muslim_ids.pkl".format(state.split()[-1])), "rb"))
    file_list = [_file for _file in os.listdir(_path) if ".csv" in _file]
    for _file in file_list:
        df = pickle.load(open(os.path.join(_path, _file), "rb"))
        df["muslim"] = df["id"].apply(lambda x : 1 if x in muslim_ids else 0)
        pickle.dump(df, open(os.path.join(_path, _file), "wb"))
    print("{} completed".format(state.split()[-1]))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

Andhra_Pradesh completed
time taken: 0.58 mins
time taken: 0.58 mins


In [56]:
#creating a dataframe for every state in "Cleaned Data" which will contain the columns "Date", "State",
#"total tweets", "total negative tweets", "muslim tweets", "muslim negative tweets"

In [15]:
t = time()
state_list = os.listdir("Cleaned Data")
for state in state_list:
    t_1 = time()
    _path = os.path.join("Cleaned Data", state)
    file_list = [file for file in os.listdir(_path) if ".csv" in file]
    state_details = []
    for file in file_list:
        day_details = {}
        day_details["date"] = re.search(r'2020-[0-9]{2}-[0-9]{2}', file)[0]
        df = pickle.load(open(os.path.join(_path, file), "rb"))
        day_details["total tweets"] = df.shape[0]
        try:
            day_details["total negative tweets"] = df.groupby("polarity_new").count()["id"][0]
        except:
            day_details["total negative tweets"] = 0
        muslim_df = df[df["muslim"] == 1]
        day_details["muslim tweets"] = muslim_df.shape[0]
        try:
            day_details["muslim negative tweets"] = muslim_df.groupby("polarity_new").count()["id"][0]
        except:
            day_details["muslim negative tweets"] = 0
        day_details["state"] = state.split()[-1]
        state_details.append(day_details)
    state_df = pd.DataFrame.from_dict(state_details)
    pickle.dump(state_df, open(os.path.join(_path, "{} tweet_details.pkl".format(state.split()[-1])), "wb"))
    print("{} completed".format(state.split()[-1]))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

Andhra_Pradesh completed
time taken: 0.01 mins
time taken: 0.01 mins


In [78]:
#going through all states in the folder "Cleaned Data" and loading the dataframe that we made in the previous step
#putting all the dataframes in one dataframe which will contain 130 days worth of data for each state

In [9]:
state_list = os.listdir("Cleaned Data")
df_list = []
for state in state_list:
    _path = os.path.join("Cleaned Data", state)
    df = pickle.load(open(os.path.join(_path, "{} tweet_details.pkl".format(state.split()[-1])), "rb"))
    df_list.append(df)

In [12]:
national_tweets_df = pd.concat(df_list, ignore_index = True)

In [85]:
#creating column containing the day the tweet was made

In [13]:
national_tweets_df["datetime"] = pd.to_datetime(national_tweets_df["date"], format = "%Y/%m/%d")
national_tweets_df["day"] = national_tweets_df["datetime"].apply(lambda x : x.day_name())

In [16]:
national_tweets_df.head()

,date,total tweets,total negative tweets,muslim tweets,muslim negative tweets,state,datetime,day
0,2020-01-22,802,1,5,1,Andhra_Pradesh,2020-01-22,Wednesday
1,2020-01-23,880,1,16,1,Andhra_Pradesh,2020-01-23,Thursday
2,2020-01-24,874,0,7,0,Andhra_Pradesh,2020-01-24,Friday
3,2020-01-25,812,1,7,1,Andhra_Pradesh,2020-01-25,Saturday
4,2020-01-26,936,1,16,1,Andhra_Pradesh,2020-01-26,Sunday


In [91]:
national_tweets_df.shape

(130, 8)

In [88]:
#saving the national_tweets_df dataframe on disk

In [89]:
pickle.dump(national_tweets_df, open("national tweet_details.pkl", "wb"))